In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evalml

In [3]:
import evalml

In [4]:
#Data Read
df=pd.read_csv('../input/parkinsons-disease-classification/pd_speech_features.csv',index_col=0, delimiter=',', skiprows=1)

In [5]:
X=df.drop('class',axis=1)
y=df['class']

In [6]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='binary')

In [7]:
X_train.head()

In [8]:
evalml.problem_types.ProblemTypes.all_problem_types

In [9]:
from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')
automl.search()

In [10]:
automl.rankings

In [11]:
automl.best_pipeline

In [12]:
best_pipeline=automl.best_pipeline

In [13]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

In [14]:
### Evaluate on hold out data
best_pipeline.score(X_test, y_test, objectives=["auc","f1","Precision","Recall"])

In [15]:
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='binary',
                          objective='auc',
                          additional_objectives=['f1', 'precision'],
                          max_batches=1,
                          optimize_thresholds=True)

automl_auc.search()

In [16]:

automl_auc.rankings

In [17]:

automl_auc.describe_pipeline(automl_auc.rankings.iloc[0]["id"])

In [18]:
best_pipeline_auc = automl_auc.best_pipeline

In [19]:
# get the score on holdout data
best_pipeline_auc.score(X_test, y_test,  objectives=["auc"])

In [20]:
best_pipeline.save("model.pkl")

In [21]:

check_model=automl.load('model.pkl')

In [23]:

check_model.predict_proba(X_test).head()

In [24]:

check_model.predict_proba(X_test).tail()